In [2]:

#creates spider prompts which serves as input in the finetune dataset of spider.
#{input: prompt, output:golden query}
import os 
import json


In [3]:
#fusion of spider and wiki-sql
from datasets import load_dataset
dataset = load_dataset("b-mc2/sql-create-context", split="train")
dataset[1]

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'context': 'CREATE TABLE head (name VARCHAR, born_state VARCHAR, age VARCHAR)',
 'answer': 'SELECT name, born_state, age FROM head ORDER BY age',
 'question': 'List the name, born state and age of the heads of departments ordered by age.'}

In [ ]:
import json



# data=[]
# for queryIdx in range(len(dataset)):
#     # data.append({
#     #     "input":generate_prompt(query),
#     #     "output":query["answer"]
#     # })
#     with open(f'/workspace/data/finetuning/spider/input/query_{queryIdx+1}.txt',"w") as input_file:
#         input_file.write(generate_prompt(dataset[queryIdx]))
#     with open(f'/workspace/data/finetuning/spider/output/query_{queryIdx+1}.txt',"w") as output_file:
#         output_file.write(dataset[queryIdx]["answer"])
#     input_file.close()
#     output_file.close()


#json_string = json.dumps(data, indent=4)

In [4]:

data=[]
prompt=f'''
[SYSTEM]:"You are an expert Text-to-SQL generator assistant. Your goal is to provide correct SQL queries to the given text description. Your output only contains the SQL code. No explanation or introductory sentences surrounding the SQL response is needed. You are given schema information. Here is the schema information: 
{"{schema_info}"}. Here are the 5 critical rules for the interactions you must abide: <rules> 1. Do not wrap the generated SQL code within SQL code markdown format. Also, do not include the SQL keyword in the beginning of the response. 2. If I don't tell you to find the limited set of results, limit to 100. 3. Only use table and columns from the list provided 4. When performing aliasing, make sure to refer the aliased tables as alias.column_name and not as alias_column_name. 5. For US state names, use abbreviated forms. For example, for South Dakota state, use SD.</rules> 

"Here is the user question:"[/SYSTEM]
[HUMAN]:{"{user_input}"}[/HUMAN]
'''

for queryIdx in range(len(dataset)):
    user_input=prompt.format(schema_info=dataset[queryIdx]["context"],user_input=dataset[queryIdx]["question"])
    output=dataset[queryIdx]["answer"]
    data.append({
        "input":user_input,
        "output":output,
        "text": f'''\n{user_input}\n[SEP] {output}'''
    })
    


json_string = json.dumps(data, indent=4)




# Write the JSON string to a file
with open('/workspace/data/finetuning/spider/spider_wikisql_dataset.json', 'w') as file:
    file.write(json_string)

In [3]:
#create modified dataset including prompt for fine-tuning 
import json
file_path = "/workspace/data/finetuning/dataset/sqlize-dataset.json"

with open(file_path, 'r') as file:
    sqlize_data = json.load(file)




In [4]:
print(len(sqlize_data))


1089


In [5]:
finetuned_prompt=f'''
[SYSTEM]:"You are an expert Text-to-SQL generator assistant. Your goal is to provide correct SQL queries to the given text description. Your output only contains the SQL code. No explanation or introductory sentences surrounding the SQL response is needed. You are given schema information. Here is the schema information: 
{"{schema_info}"}. Here are the 5 critical rules for the interactions you must abide: <rules> 1. Do not wrap the generated SQL code within SQL code markdown format. Also, do not include the SQL keyword in the beginning of the response. 2. If I don't tell you to find the limited set of results, limit to 100. 3. Only use table and columns from the list provided 4. When performing aliasing, make sure to refer the aliased tables as alias.column_name and not as alias_column_name. 5. For US state names, use abbreviated forms. For example, for South Dakota state, use SD.</rules> 

"Here is the user question:"[/SYSTEM]
[HUMAN]:{"{user_input}"}[/HUMAN]
'''

In [6]:
finetuned_data=[]
for queryIdx in range(len(sqlize_data)):
    user_input=finetuned_prompt.format(schema_info=sqlize_data[queryIdx]["schema"],user_input=sqlize_data[queryIdx]["nlq"])
    output=sqlize_data[queryIdx]["sql"]
    finetuned_data.append({
        "id":sqlize_data[queryIdx]["id"],
        "input":user_input,
        "output":output,
        "text": f'''\n{user_input}\n[SEP] {output}'''
    })

json_string = json.dumps(finetuned_data, indent=4)




# Write the JSON string to a file
with open('/workspace/data/finetuning/dataset/sqlize-finetuned-dataset.json', 'w') as file:
    file.write(json_string)   